In [2]:
import numpy as np
import matplotlib.pyplot as plt

class neuralNetwork:
    #初始化神經網路
    def __init__(self, inputnodes, hidden1nodes, hidden2nodes, outputnodes, learning_rate):
        self.inodes=inputnodes#輸入層節點個數
        self.h1nodes=hidden1nodes#第一層隱藏層節點個數
        self.h2nodes=hidden2nodes#第一層隱藏層節點個數
        self.onodes=outputnodes#輸出層節點個數
        self.lr=learning_rate#學習率
        #輸入層到第一層隱藏層的權重
        #self.wih1=np.random.normal(0.0, pow(self.h1nodes,-0.5), (self.h1nodes, self.inodes))
        #第一層隱藏層到第二層隱藏層的權重
        #self.wh1h2=np.random.normal(0.0, pow(self.h2nodes,-0.5), (self.h2nodes, self.h1nodes))
        #第二層隱藏層到輸出層的權重
        #self.wh2o=np.random.normal(0.0, pow(self.onodes,-0.5), (self.onodes, self.h2nodes))
        #另一種方法
        self.wih1=np.random.rand(self.h1nodes, self.inodes)-0.5
        self.wh1h2=np.random.rand(self.h2nodes, self.h1nodes)-0.5
        self.wh2o=np.random.rand(self.onodes, self.h2nodes)-0.5
        pass
    
    def train(self, inputs_list, targets_list):#訓練神經網路模型
        inputs=np.array(inputs_list, ndmin=2).T#這是行向量
        targets=np.array(targets_list, ndmin=2).T#這是行向量
        
        hidden1_inputs=np.dot(self.wih1, inputs)#第一層隱藏層的輸入，這是行向量
        hidden1_outputs=(1/(1+np.exp(-1*hidden1_inputs)))#第一層隱藏層的輸出，這是行向量
        
        hidden2_inputs=np.dot(self.wh1h2, hidden1_outputs)#第二層隱藏層的輸入，這是行向量
        hidden2_outputs=(1/(1+np.exp(-1*hidden2_inputs)))#第二層隱藏層的輸出，這是行向量
        
        final_inputs=np.dot(self.wh2o, hidden2_outputs)#輸出層的輸入，這是行向量
        final_outputs=(1/(1+np.exp(-1*final_inputs)))#輸出層的輸出，這是行向量
        
        
        output_errors=(targets-final_outputs)*final_outputs*(1-final_outputs)#輸出層輸出誤差，這是行向量
        hidden2_errors=(np.dot(self.wh2o.T, output_errors))*(hidden2_outputs*(1-hidden2_outputs))#第二層隱藏層輸出的誤差，這是行向量
        hidden1_errors=(np.dot(self.wh1h2.T, hidden2_errors))*(hidden1_outputs*(1-hidden1_outputs))#第一層隱藏層輸出的誤差，這是行向量
        
        #更新第二隱藏層至輸出層的權重
        #output_errors*final_outputs*(1.0-final_outputs)為行向量對應元素相乘
        #hidden_outputs.T，轉成列向量，dot完才會變成與權重相當大小的矩陣
        self.wh2o=self.wh2o+self.lr*np.dot(output_errors,hidden2_outputs.T)
        #更新第一隱藏層至第二隱藏層的權重
        self.wh1h2=self.wh1h2+self.lr*np.dot(hidden2_errors,hidden1_outputs.T)
        #更新輸出層至隱藏層的權重
        self.wih1=self.wih1+self.lr*np.dot(hidden1_errors,inputs.T)
        pass
    
    def query(self, inputs_list):#丟輸入進來，回傳輸出值
        inputs=np.array(inputs_list, ndmin=2).T#這是行向量
        
        hidden1_inputs=np.dot(self.wih1, inputs)#第一層隱藏層的輸入，這是行向量
        hidden1_outputs=(1/(1+np.exp(-1*hidden1_inputs)))#第一層隱藏層的輸出，這是行向量
        
        hidden2_inputs=np.dot(self.wh1h2, hidden1_outputs)#第二層隱藏層的輸入，這是行向量
        hidden2_outputs=(1/(1+np.exp(-1*hidden2_inputs)))#第二層隱藏層的輸出，這是行向量
        
        final_inputs=np.dot(self.wh2o, hidden2_outputs)#輸出層的輸入，這是行向量
        final_outputs=(1/(1+np.exp(-1*final_inputs)))#輸出層的輸出，這是行向量
        
        return final_outputs


inputnodes=784#因為圖片像素為28*28=784
hidden1nodes=100#沒有最佳選擇方法，但不須高於inputnodes，在簡短形式下網路應該有能力發現特徵
hidden2nodes=100
outputnodes=10#數字=0~9
learning_rate=0.5
#實體化一個神經網路
n=neuralNetwork(inputnodes, hidden1nodes, hidden2nodes, outputnodes, learning_rate)
#載入訓練集
training_data_file=open("mnist_train.csv",'r')
training_data_list=training_data_file.readlines()
training_data_file.close()
#開始訓練神經網路
epoch=1
for e in range(epoch):
    for record in training_data_list:
        all_value=record.split(',')#一次取一條數據
        inputs=(np.asfarray(all_value[1:])/255*0.99)+0.01#輸入要scale成0.01~1
        targets=np.zeros(outputnodes)+0.01
        targets[int(all_value[0])]=0.99#目標值要符合sigmoid的值域
        n.train(inputs, targets)
        pass
    pass
#=================測試區=============================

acc=0
err=0
test_data_file=open("mnist_test.csv",'r')
test_data_list=test_data_file.readlines()
test_data_file.close()
for record in test_data_list:
    all_value=record.split(',')
    correct_label=int(all_value[0])
    inputs=(np.asfarray(all_value[1:])/255*0.99)+0.01
    outputs=n.query(inputs)
    predict_label=np.argmax(outputs)#回傳最大值所在的索引
    #print("correct_label:%s , predict_label:%s" %(correct_label,predict_label))
    if(correct_label==predict_label):
        acc=acc+1
    elif(correct_label!=predict_label):
        err=err+1
    pass
print("正確次數:%s , 錯誤次數:%s" %(acc,err))
accuracy=acc/(acc+err)*100
print("準確率:%s" %accuracy)

正確次數:9361 , 錯誤次數:639
準確率:93.61
